# **Instance Segmentation on CRAL**

Tutorial on how to use CRAL library for instance segmentation using MaskRCNN.

In [ ]:
import cral
import tempfile

In [ ]:
import tensorflow as tf
tf.python.control_flow_ops = tf

**Configure CRAL**

CRAL needs to be configured with cral config after installing the library. After running this, you will be asked for the email and password. Only after this will the tracking of the runs be possible

In [ ]:
!cral config

**Experiment Tracking**

This is a mandatory step to be run before training. It provides you statistics on training data and logs all the metrics, losses etc. in an online dashboard making it easier to evaluate the model and compare between different runs, which can help you set the right hyperparameters.

In [ ]:
from cral.pipeline.instance_segmentation_pipeline import InstanceSegPipe
from cral.tracking import set_experiment
set_experiment("your-experiment-id")

new_pipe = InstanceSegPipe()

**Path to Dataset**

Supported formats : coco

In [ ]:
train_images_path = '/path/to/images/directory/'
train_anno_path = '/path/to/annotations'

**Add data and create tfrecords**

In [ ]:
new_pipe.add_data(
    train_images_dir=train_images_path,
    train_anno_dir=train_anno_path,
    annotation_format='coco',
    split=0.2)

meta_info = new_pipe.lock_data()

In [ ]:
meta_info

**For MaskRCNN**

Supported Backbones : resnet50, resnet101



In [ ]:
from cral.models.instance_segmentation.MaskRCNN import MaskRCNNConfig

maskrcnn_config = MaskRCNNConfig(
    height=1024, 
    width=1024, classes=int(meta_info['num_classes']))

new_pipe.set_algo(
    feature_extractor='resnet101',
    config=maskrcnn_config, weights = 'coco')

# Train


Now that you have set the model, you can start training. Using train is very similar to the fit function in tensorflow.keras and it accepts many of the same arguments like epochs, batch_size etc. All your loss and metrics will be streamed from here onto track.segmind.com automatically where you can see and evaluate your model easily.

In [ ]:
new_pipe.train(
    num_epochs=50,
    snapshot_prefix='maskrcnn',
    snapshot_path=tempfile.gettempdir(),
    snapshot_every_n=5
    )

**Prediction**

After training you can use the model to get predictions on images that maybe or may not be a part of your initial dataset.

Calling prediction_model will give you a function that can be used to get predictions on images.

In [ ]:
checkpoint_file = '/tmp/maskrcnn_final.h5'
pred_func = new_pipe.prediction_model(checkpoint_file= checkpoint_file)


**For FasterRCNN**

Demonstrating the predicted results

In [ ]:
from cral.models.instance_segmentation import annotate_image
import os

import skimage
real_test_dir = '/path/to/test/images/'
image_paths = []
for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in image_paths:
  mask_array = pred_func(image_path)
  annotate_image(image_path, mask_array['rois'], mask_array['masks'], mask_array['class_ids'], mask_array['scores'], figsize=(5,5))

